## Deliverable 2. Create a Customer Travel Destinations Map.

In [17]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

In [18]:
# Import the API keys from config
import sys
import os

# get name of this directory
current = os.path.dirname(os.path.realpath('Weather_Database'))

# get name of the parent directory
parent = os.path.dirname(current)

# adding parent directory to the sys.path
sys.path.append(parent)

# importing config
from config import weather_api_key

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [19]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,49.08,81,75,4.41,broken clouds
1,1,Rikitea,PF,-23.1203,-134.9692,73.53,79,100,12.37,light rain
2,2,Clyde River,CA,70.4692,-68.5914,21.96,85,100,26.78,light snow
3,3,Ribeira Grande,PT,38.5167,-28.7000,70.20,88,75,24.16,broken clouds
4,4,Cap Malheureux,MU,-19.9842,57.6142,71.47,76,0,4.00,clear sky


In [20]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [21]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

# Filters cities based on user input
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,-23.1203,-134.9692,73.53,79,100,12.37,light rain
3,3,Ribeira Grande,PT,38.5167,-28.7000,70.20,88,75,24.16,broken clouds
4,4,Cap Malheureux,MU,-19.9842,57.6142,71.47,76,0,4.00,clear sky
8,8,Valdez,EC,1.2492,-78.9831,79.77,77,20,10.29,few clouds
11,11,Hilo,US,19.7297,-155.0900,78.62,70,90,4.61,overcast clouds
15,15,Winton,US,37.3894,-120.6133,73.51,80,90,6.91,haze
17,17,Saint George,US,37.1041,-113.5841,77.47,34,1,1.01,clear sky
18,18,Atuona,PF,-9.8000,-139.0333,77.85,64,31,16.46,scattered clouds
19,19,Mana,GF,5.6592,-53.7767,84.72,63,95,9.84,overcast clouds
20,20,Samarai,PG,-10.6167,150.6667,80.29,81,48,9.31,scattered clouds


In [22]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                286
City                   286
Country                283
Lat                    286
Lng                    286
Max Temp               286
Humidity               286
Cloudiness             286
Wind Speed             286
Current Description    286
dtype: int64

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,73.53,light rain,-23.1203,-134.9692,
3,Ribeira Grande,PT,70.20,broken clouds,38.5167,-28.7000,
4,Cap Malheureux,MU,71.47,clear sky,-19.9842,57.6142,
8,Valdez,EC,79.77,few clouds,1.2492,-78.9831,
11,Hilo,US,78.62,overcast clouds,19.7297,-155.0900,
15,Winton,US,73.51,haze,37.3894,-120.6133,
17,Saint George,US,77.47,clear sky,37.1041,-113.5841,
18,Atuona,PF,77.85,scattered clouds,-9.8000,-139.0333,
19,Mana,GF,84.72,overcast clouds,5.6592,-53.7767,
20,Samarai,PG,80.29,scattered clouds,-10.6167,150.6667,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Adding location to params
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [31]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [32]:
# 8a. Create the output File (CSV)
output_data_file = './WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
